Trying out sktime for our time series

In [1]:
import math
import pandas as pandas
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn import preprocessing
from statistics import mean
from numpy.random import shuffle

import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

data_248_original = pandas.read_json("../data/device_id_248.json")
data_249_original = pandas.read_json("../data/device_id_249.json")
data_248 = data_248_original.copy()
data_249 = data_249_original.copy()
data_249

,tot1,tot2,analog2
2018-11-23 08:55:00,0.43,0.00,1.11
2018-11-23 08:56:00,0.89,0.00,1.11
2018-11-23 08:57:00,1.38,0.00,1.11
2018-11-23 08:58:00,1.90,0.00,1.11
2018-11-23 08:59:00,2.45,0.00,1.11
...,...,...,...
2020-04-01 08:35:00,797188.56,-293.97,0.82
2020-04-01 08:36:00,797192.50,-293.97,0.81
2020-04-01 08:37:00,797195.06,-293.97,0.81
2020-04-01 08:38:00,797197.62,-293.97,0.81


## Tranformed time into tabular data

In [2]:
data_249['day'] = [i.day for i in data_249.index]
data_249['month'] = [i.month for i in data_249.index]
data_249['year'] = [i.year for i in data_249.index]
data_249['hour'] = [i.hour for i in data_249.index]
data_249['minute'] = [i.minute for i in data_249.index]

data_249_index = data_249.reset_index()
del data_249_index["index"]
data_249_index

,tot1,tot2,analog2,day,month,year,hour,minute
0,0.43,0.00,1.11,23,11,2018,8,55
1,0.89,0.00,1.11,23,11,2018,8,56
2,1.38,0.00,1.11,23,11,2018,8,57
3,1.90,0.00,1.11,23,11,2018,8,58
4,2.45,0.00,1.11,23,11,2018,8,59
...,...,...,...,...,...,...,...,...
705853,797188.56,-293.97,0.82,1,4,2020,8,35
705854,797192.50,-293.97,0.81,1,4,2020,8,36
705855,797195.06,-293.97,0.81,1,4,2020,8,37
705856,797197.62,-293.97,0.81,1,4,2020,8,38
